In [1]:
"""
In this third example, we will take a closer look at the model.

There exist a wealth of different models all with various strengths and weaknesses.

Diving into model details is beyond the scope of this workshop,
but suffice to say that model architecture is constantly evolving and unless you are an expert you are likely best off just using some well performing neural network for a problem similar to the one you are interested in.

Common to all neural networks is that they consist of some amount of trainable parameters and in general the more parameters a model has, the more memory it requires to train/run and the longer it takes.
"""

'\nIn this third example, we will take a closer look at the model.\n\nThere exist a wealth of different models all with various strengths and weaknesses.\n\nDiving into model details is beyond the scope of this workshop,\nbut suffice to say that model architecture is constantly evolving and unless you are an expert you are likely best off just using some well performing neural network for a problem similar to the one you are interested in.\n\nCommon to all neural networks is that they consist of some amount of trainable parameters and in general the more parameters a model has, the more memory it requires to train/run and the longer it takes.\n'

In [2]:
from config.efficient_net_b2_pretrained import Configuration
# load configuration information
configuration = Configuration()


In [4]:
import torch
import torch.nn as nn
import torchvision

"""
In this project we have provided a simple U-net convolutional model, as well as a few models from torchvisions modelhub.
"""

def model_loader_wrapper(model_name, model_weights=None, device='cpu', **kwargs):
    """
    Selects the model we want to use.

    Note that we have both premade/pretrained models as well as our own custom models in here.
    """

    if model_name == "resnet50":
        model = torchvision.models.resnet50(weights=model_weights)
    elif model_name == "efficient_net_b2":
        model = torchvision.models.efficientnet_b2(weights=model_weights)
        model.classifier[1] = torch.nn.Linear(1408,2,bias=True)
    elif model_name == 'unet':
        model = UNet()
    else:
        raise NotImplementedError(f"{model_name} has not been implemented.")

    total_params = sum(p.numel() for p in model.parameters())
    print(f'{model_name} loaded, containing {total_params/1000000:2.2f}M trainable parameters.')
    model.to(device=device)

    return model

class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class Up(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Up, self).__init__()
        self.up_scale = nn.ConvTranspose2d(in_ch, out_ch, kernel_size=2, stride=2)

    def forward(self, x1, x2):
        x2 = self.up_scale(x2)

        diffY = x1.size()[2] - x2.size()[2]
        diffX = x1.size()[3] - x2.size()[3]

        x2 = torch.functional.pad(x2, [diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return x


class DownLayer(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(DownLayer, self).__init__()
        self.pool = nn.MaxPool2d(2, stride=2, padding=0)
        self.conv = DoubleConv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(self.pool(x))
        return x


class UpLayer(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(UpLayer, self).__init__()
        self.up = Up(in_ch, out_ch)
        self.conv = DoubleConv(in_ch, out_ch)

    def forward(self, x1, x2):
        a = self.up(x1, x2)
        x = self.conv(a)
        return x


class UNet(nn.Module):
    def __init__(self, dimensions=2):
        super(UNet, self).__init__()
        self.conv1 = DoubleConv(3, 64)
        self.down1 = DownLayer(64, 128)
        self.down2 = DownLayer(128, 256)
        self.down3 = DownLayer(256, 512)
        self.down4 = DownLayer(512, 1024)
        self.up1 = UpLayer(1024, 512)
        self.up2 = UpLayer(512, 256)
        self.up3 = UpLayer(256, 128)
        self.up4 = UpLayer(128, 64)
        self.avgpool = torch.nn.AdaptiveAvgPool2d(output_size=1)
        self.classifier = torch.nn.Sequential(torch.nn.Dropout(p=0.3, inplace=True), torch.nn.Linear(64,dimensions,bias=True))

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x1_up = self.up1(x4, x5)
        x2_up = self.up2(x3, x1_up)
        x3_up = self.up3(x2, x2_up)
        x4_up = self.up4(x1, x3_up)
        y = self.avgpool(x4_up)
        output = self.classifier(y[:,:,0,0])
        return output


In [5]:
from src.dataloader import load_data_wrapper
from src.losses import select_loss_fnc
from src.optimizer import optimize_model
from src.viz import eval_unlabelled_images, plot_loss_and_accuracy


dataloaders = load_data_wrapper(**configuration)
model = model_loader_wrapper(**configuration) # Here we build our model
optimizer = torch.optim.Adam(model.parameters(), lr=configuration['lr'])
loss_fnc = select_loss_fnc(**configuration)


efficient_net_b2 loaded, containing 7.70M trainable parameters.


In [7]:
%pip install torchsummary
from torchsummary import summary

"""
Next we need to train the model.
The overall training will be the focus of the 4th exercise, so we won't dive into that yet, but we will take a quick step inside the training and focus on how the data propagates through the model.
"""

# this is how we would train the model
# losses, accuracies = optimize_model(model, dataloaders, optimizer, loss_fnc, **configuration)

# for now, we will focus on the summary:
summary(model, (3, 256, 256), )

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


NameError: name 'device' is not defined

# A few questions to consider:

1. Based on the shape of the data as it changed through the forward function of the model what do you think is happening?
2. When we use a pretrained model and change the last layers to fit our particular needs, do we then need to retrain the whole model?
